# <u> <font color = "teal"> Tareas:</font> </u>
## <font color = "teal"> * Detectar archivos con problemas de georrefercia </font> 
## <font color = "teal"> * Implementar procesos de renombre </font> 

---


## <font color = "lime"> .csv de Argendata: cada uno de sus encodings y delimiters </font>

No todos los .csv que contempla Argendata tienen el mismo enconding y delimiter. Por eso antes de comenzar a trabajar voy a armar el siguiente input para la tarea: dataframe que contenga 4 columnas que sean 'TOPICO', 'archivo_csv', 'encoding' y delimiter'.

Me gustaría algo explicito que por cada carpeta (topico) dentro de /home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata encuentre por cada uno de sus .csv el encoding y delimiter específico para abrir cada .csv. Y que el resultado final sea un dataframe que contenga 4 columnas que sean 'TOPICO', 'archivo_csv', 'encoding' y delimiter'.

### <font color = "red"> ¿Meter este proceso en QA? Para ya tener el respectivo encoding delimiter de cada.csv </font>

In [ ]:
import os
import glob
import chardet
import csv
import pandas as pd

# ruta de tabajo
base_dir = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata'

# lista para recolectar los datos
data = []

# tamanio del fragmento a leer (en este caso 100000 bytes)
sample_size = 100000

for topico in os.listdir(base_dir): # ejemplo SEBACO
    topico_path = os.path.join(base_dir, topico) #ejemplo /home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata/SEBACO

    # ignorar archivos sueltos como LICENSE y README.md
    if not os.path.isdir(topico_path):
        continue

    for archivo in os.listdir(topico_path):
        if archivo.endswith('.csv'):
            archivo_path = os.path.join(topico_path, archivo)

            # leer un fragmento del .csv para análisis
            with open(archivo_path, 'rb') as f:
                raw_sample = f.read(sample_size)

            # detectar encoding
            encoding_detected = 'utf-8'  # por defecto
            try:
                encoding_info = chardet.detect(raw_sample)
                if encoding_info and encoding_info['encoding']:
                    encoding_detected = encoding_info['encoding']
            except:
                pass

            # decodificar el fragmento
            try:
                sample = raw_sample.decode(encoding_detected, errors='replace')
            except:
                sample = raw_sample.decode('utf-8', errors='replace') 

            # detectar delimitador
            delimiter_detected = ','
            try:
                sniffer = csv.Sniffer()
                dialect = sniffer.sniff(sample)
                delimiter_detected = dialect.delimiter
            except:
                pass

            # agregar al resultado
            data.append({
                'TOPICO': topico,
                'archivo_csv': archivo,
                'encoding': encoding_detected,
                'delimiter': delimiter_detected
            })

# crear el DataFrame
df_resultado = pd.DataFrame(data)

# mostrar resultado
# print(df_resultado)

# guardar como csv el archivo csvTopicoArgendata_enciding_and_delimiters
# path_df_encoding_delimiters = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/'
# df_resultado.to_csv(path_df_encoding_delimiters + 'csvTopicosArgendata_codificacion_y_delimitadores.csv', index=False)


Ahora que tengo 'csvTopicosArgendata_codificacion_y_delimitadores.csv, voy a enfocarme en aquellas columnas que tengan datos de georreferencia

In [ ]:
Ahora que ya tengo todos los csv de todos los tópicos de Argendata/data junto con sus columnas, voy a enfocarme en aquellas que tengan información de georreferencia. Para eso voy a usar dos inputs dados y algunas consideraciones:
- Joan me pasó columnas que hasta el momento se sabe que son de georef -> georef_keys.txt
- voy a usar el geonomeclador de argendata, la pestaña curada export (geonomeclador_export.csv)
- voy a tener en cuenta solo aquellas comunnas de georeferencia en español: geonomeclador_export.csv para lo que es las columnas "name_long" y "name_short" solo considera el español
- de las columnas a investigar y buscar si corresponden a georeferencia, voy a quedarme con aquellas que: el 80% de su contenido aparezca en cada csv que busco

Que voy a considerar como observaciones:
- aquellas columnas, de sus respectivos .csv y tópicos que no estén en español
- aquellos nombres/códigos que deberían ser cierta "nombre/codigo" y no lo son